In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model

df = pd.read_csv("Data.csv", sep=";", skiprows=5)
print(df.head())

# Rename columns
df.rename(columns={
    "Unnamed: 0": "Date",
    "Dow Jones Global": "DJIA",
    "FTSE UK": "FTSE100",
    "Euronext": "CAC40"
}, inplace=True)

print(df)

# Convert date
df['Date'] = pd.to_datetime(df['Date'].astype(str), format="%Y%m%d")

df

# 0a - convert European comma decimals to float
num_cols = ["DJIA", "FTSE100", "CAC40"]
for c in num_cols:
    df[c] = df[c].astype(str).str.replace(",", ".", regex=False).astype(float)

df

   Unnamed: 0 Dow Jones Global  FTSE UK    Euronext
0    20090806      16090,17457  2929,07  6560,50517
1    20090807      16288,02074  2954,69  6642,19415
2    20090810      16232,18766  2948,84  6610,88828
3    20090811      16069,88812  2917,08  6519,66241
4    20090812      16287,93145   2950,9  6615,98017
          Date         DJIA      FTSE100        CAC40
0     20090806  16090,17457      2929,07   6560,50517
1     20090807  16288,02074      2954,69   6642,19415
2     20090810  16232,18766      2948,84   6610,88828
3     20090811  16069,88812      2917,08   6519,66241
4     20090812  16287,93145       2950,9   6615,98017
...        ...          ...          ...          ...
4257  20251201  120252,1799  11130,29467  26099,66913
4258  20251202  120785,5729  11129,46209  26055,97937
4259  20251203  121824,7173  11118,29503  26097,29305
4260  20251204  121779,6052  11141,96342  26208,99831
4261  20251205  122044,4041  11091,64282  26185,45931

[4262 rows x 4 columns]


,Date,DJIA,FTSE100,CAC40
0,2009-08-06,16090.17457,2929.07000,6560.50517
1,2009-08-07,16288.02074,2954.69000,6642.19415
2,2009-08-10,16232.18766,2948.84000,6610.88828
3,2009-08-11,16069.88812,2917.08000,6519.66241
4,2009-08-12,16287.93145,2950.90000,6615.98017
...,...,...,...,...
4257,2025-12-01,120252.17990,11130.29467,26099.66913
4258,2025-12-02,120785.57290,11129.46209,26055.97937
4259,2025-12-03,121824.71730,11118.29503,26097.29305
4260,2025-12-04,121779.60520,11141.96342,26208.99831


In [2]:
df = df.sort_values("Date").reset_index(drop=True)

# simple percentage returns (Hull-style)
df["r_DJIA"] = df["DJIA"].pct_change()
df["r_FTSE"] = df["FTSE100"].pct_change()
df["r_CAC"]  = df["CAC40"].pct_change()

df = df.dropna().reset_index(drop=True)

print(df[["Date", "r_DJIA", "r_FTSE", "r_CAC"]].head())

        Date    r_DJIA    r_FTSE     r_CAC
0 2009-08-07  0.012296  0.008747  0.012452
1 2009-08-10 -0.003428 -0.001980 -0.004713
2 2009-08-11 -0.009999 -0.010770 -0.013799
3 2009-08-12  0.013568  0.011594  0.014773
4 2009-08-13  0.003907  0.008204  0.004891


In [3]:
# EWMA parameter (RiskMetrics / Hull 10.6)
lam = 0.94  

def ewma_sigma(r, lam=0.94):
    r = r.dropna().reset_index(drop=True)
    T = len(r)
    var = np.empty(T)
    
    # starting value: variance over whole sample period
    var[0] = r.var(ddof=1)
    
    # recursion: sigma_t^2 = λ sigma_{t-1}^2 + (1-λ) u_{t-1}^2
    for t in range(1, T):
        var[t] = lam * var[t-1] + (1 - lam) * (r.iloc[t-1] ** 2)
    
    return np.sqrt(var)

df["sig_EWMA_DJIA"] = ewma_sigma(df["r_DJIA"], lam)
df["sig_EWMA_FTSE"] = ewma_sigma(df["r_FTSE"], lam)
df["sig_EWMA_CAC"]  = ewma_sigma(df["r_CAC"],  lam)

# σ_n = last available volatility estimate for each index
sigma_n = df[["sig_EWMA_DJIA", "sig_EWMA_FTSE", "sig_EWMA_CAC"]].iloc[-1]
print("EWMA σ_n (last day):")
print(sigma_n)

EWMA σ_n (last day):
sig_EWMA_DJIA    0.007664
sig_EWMA_FTSE    0.005427
sig_EWMA_CAC     0.006768
Name: 4260, dtype: float64


In [4]:
def garch_sigma_last(r):
    # GARCH(1,1) uses returns in percent by convention
    r_pct = r * 100
    
    # mean="zero" matches Hull (no mean term for daily returns)
    model = arch_model(r_pct, mean="zero", vol="GARCH", p=1, q=1)
    res = model.fit(disp="off")
    
    # conditional volatility in percent → convert back to daily decimal
    sigma = res.conditional_volatility / 100
    return sigma.iloc[-1], res.params   # σ_n, parameters (ω, α, β)

# GARCH for each index
sigma_garch_djia, params_djia = garch_sigma_last(df["r_DJIA"])
sigma_garch_ftse, params_ftse = garch_sigma_last(df["r_FTSE"])
sigma_garch_cac,  params_cac  = garch_sigma_last(df["r_CAC"])

print("GARCH σ_n (last day):")
print("DJIA :", sigma_garch_djia)
print("FTSE :", sigma_garch_ftse)
print("CAC40:", sigma_garch_cac)

print("\nGARCH parameters (ω, α, β):")
print("DJIA params:", params_djia)
print("FTSE params:", params_ftse)
print("CAC params :", params_cac)

GARCH σ_n (last day):
DJIA : 0.007663753344629982
FTSE : 0.005871445100076458
CAC40: 0.007202184465880457

GARCH parameters (ω, α, β):
DJIA params: omega       0.033924
alpha[1]    0.144219
beta[1]     0.818498
Name: params, dtype: float64
FTSE params: omega       0.041989
alpha[1]    0.135803
beta[1]     0.815998
Name: params, dtype: float64
CAC params : omega       0.049253
alpha[1]    0.129647
beta[1]     0.838430
Name: params, dtype: float64


In [5]:
# matrix of returns: T x 3
R = df[["r_DJIA", "r_FTSE", "r_CAC"]].values
T = R.shape[0]

# equal-weighted sample mean and covariance
mu = R.mean(axis=0, keepdims=True)       # 1 x 3
R_c = R - mu                             # demeaned returns
Sigma_hist = (R_c.T @ R_c) / T           # 3 x 3 covariance matrix

print("Equal-weighted covariance matrix (historical):")
print(Sigma_hist)

# portfolio: DJIA 4000, FTSE100 3000, CAC40 3000
V0 = 4000 + 3000 + 3000
w = np.array([4000, 3000, 3000]) / V0     # weights

sigma_p = np.sqrt(w @ Sigma_hist @ w)     # 1-day portfolio volatility
print("Portfolio sigma (1 day):", sigma_p)

alpha = 0.99
z = 2.32634787404084                           # 99% standard normal quantile
phi_z = 1 / np.sqrt(2*np.pi) * np.exp(-0.5*z**2)

VaR_99 = V0 * z * sigma_p
ES_99  = V0 * sigma_p * phi_z / (1 - alpha)

print("99% 1-day VaR (hist, equal weights):", VaR_99)
print("99% 1-day ES  (hist, equal weights):", ES_99)


Equal-weighted covariance matrix (historical):
[[1.05054441e-04 5.67161519e-05 7.38812946e-05]
 [5.67161519e-05 9.23214982e-05 1.00386191e-04]
 [7.38812946e-05 1.00386191e-04 1.47863710e-04]]
Portfolio sigma (1 day): 0.009372207894190679
99% 1-day VaR (hist, equal weights): 218.0301590971926
99% 1-day ES  (hist, equal weights): 249.78941755634253


In [7]:
# =========================================
# EWMA variance–covariance (Hull 14.3.1)
# =========================================

lam = 0.94   # decay factor λ assumed (RiskMetrics / Hull)

# start from equal-weighted covariance as Σ_0
Sigma_ewma = Sigma_hist.copy()

for t in range(1, T):
    x = R[t-1].reshape(-1, 1)                 # column vector r_{t-1}
    Sigma_ewma = lam * Sigma_ewma + (1 - lam) * (x @ x.T)

print("\nEWMA covariance matrix (λ = 0.94):")
print(Sigma_ewma)

# portfolio volatility under EWMA
sigma_p_ewma = np.sqrt(w @ Sigma_ewma @ w)

VaR_ewma_99 = V0 * z * sigma_p_ewma
ES_ewma_99  = V0 * sigma_p_ewma * phi_z / (1 - alpha)

print("Portfolio sigma (EWMA, 1 day):", sigma_p_ewma)
print("99% 1-day VaR (EWMA):", VaR_ewma_99)
print("99% 1-day ES  (EWMA):", ES_ewma_99)



EWMA covariance matrix (λ = 0.94):
[[5.87368905e-05 2.52173330e-05 3.02087058e-05]
 [2.52173330e-05 2.94569368e-05 2.62126518e-05]
 [3.02087058e-05 2.62126518e-05 4.58046381e-05]]
Portfolio sigma (EWMA, 1 day): 0.005847390088557463
99% 1-day VaR (EWMA): 136.03063501203133
99% 1-day ES  (EWMA): 155.84547215932494


In [8]:
# =========================================
# Monte Carlo VaR (Hull 14.7.1)
# =========================================

n_sims = 100_000

# simulate multivariate normal daily returns with historical covariance
sim_R = np.random.multivariate_normal(
    mean=np.zeros(3),        # expected daily return ≈ 0
    cov=Sigma_hist,
    size=n_sims
)

# portfolio return and monetary loss
sim_port_ret = sim_R @ w
sim_losses   = -V0 * sim_port_ret    # ΔP = V0 * return, loss = -ΔP

# empirical 99% VaR and ES
VaR_mc_99 = np.quantile(sim_losses, 0.99)
ES_mc_99  = sim_losses[sim_losses >= VaR_mc_99].mean()

print("\nMonte Carlo 99% 1-day VaR:", VaR_mc_99)
print("Monte Carlo 99% 1-day ES :", ES_mc_99)



Monte Carlo 99% 1-day VaR: 217.5760364742812
Monte Carlo 99% 1-day ES : 250.10897206016435
